In [13]:
import glob
import re
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
from matplotlib import ticker

seaborn.set()

# you will probably have to change this
file_pattern = "./results/stdout/out_*.txt"
stdout_files = glob.glob(file_pattern)

print(stdout_files)

['./results/stdout\\out_1.txt', './results/stdout\\out_1024.txt', './results/stdout\\out_128.txt', './results/stdout\\out_16.txt', './results/stdout\\out_16384.txt', './results/stdout\\out_2.txt', './results/stdout\\out_2048.txt', './results/stdout\\out_256.txt', './results/stdout\\out_32.txt', './results/stdout\\out_4.txt', './results/stdout\\out_4096.txt', './results/stdout\\out_512.txt', './results/stdout\\out_64.txt', './results/stdout\\out_8.txt', './results/stdout\\out_8192.txt']


In [17]:
# these are the only relevant ones, at least
patterns = {
    "num_worlds": r"Running with (\d+) Worlds:",
    "allocating": r"\[INFO\] Allocating (\d+) of (\d+) bytes on device",
    "actual_program_time": r"\[BENCHMARK\] Actual program time: ([\d\.]+) ms",
    "avg_simulateKernel_time": r"\[BENCHMARK\] Average individual simulateKernel\(\) time over (\d+) samples: ([\d\.]+) ms",
    "total_kernel_time": r"\[BENCHMARK\] Total time spent in kernel before global convergence: ([\d\.]+) ms",
    "kernel_time_ratio": r"\[BENCHMARK\] Kernel time / total program time: ([\d\.]+)",
}

data = {f"before_{k}":[] for k in patterns}
data.update({f"after_{k}":[] for k in patterns})

for filename in stdout_files:
    with open(filename, 'r') as f:
        content = f.read()

    # let me put you on game: put any string in a quick cmd -> py -> len("<string>")
    sep = content.split("-" * 27)

    # there is an invalid - * 27 at the top line
    for i, partition in enumerate(sep[1:]):
        part = "before" if i == 0 else "after"
        for key, pattern in patterns.items():
            match = re.search(pattern, partition)
            if match:
                data[f"{part}_{key}"].append(match.groups())

print(data)

{'before_num_worlds': [('1',), ('1024',), ('128',), ('16',), ('16384',), ('2',), ('2048',), ('256',), ('32',), ('4',), ('4096',), ('512',), ('64',), ('8',), ('8192',)], 'before_allocating': [('2304', '25282281472'), ('2359296', '25282281472'), ('294912', '25282281472'), ('36864', '25282281472'), ('37748736', '25282281472'), ('4608', '25282281472'), ('4718592', '25282281472'), ('589824', '25282281472'), ('73728', '25282281472'), ('9216', '25282281472'), ('9437184', '25282281472'), ('1179648', '25282281472'), ('147456', '25282281472'), ('18432', '25282281472'), ('18874368', '25282281472')], 'before_actual_program_time': [('189',), ('480',), ('372',), ('261',), ('2100',), ('186',), ('538',), ('400',), ('296',), ('207',), ('618',), ('424',), ('336',), ('229',), ('1255',)], 'before_avg_simulateKernel_time': [('6575', '0.021439'), ('7724', '0.053425'), ('6625', '0.048564'), ('6597', '0.031946'), ('12368', '0.140693'), ('6558', '0.020651'), ('7899', '0.057941'), ('6662', '0.052087'), ('6616',

In [18]:
df = pd.DataFrame(data)
print(df.head())

  before_num_worlds        before_allocating before_actual_program_time  \
0              (1,)      (2304, 25282281472)                     (189,)   
1           (1024,)   (2359296, 25282281472)                     (480,)   
2            (128,)    (294912, 25282281472)                     (372,)   
3             (16,)     (36864, 25282281472)                     (261,)   
4          (16384,)  (37748736, 25282281472)                    (2100,)   

  before_avg_simulateKernel_time before_total_kernel_time  \
0               (6575, 0.021439)            (140.960938,)   
1               (7724, 0.053425)            (412.655151,)   
2               (6625, 0.048564)            (321.733337,)   
3               (6597, 0.031946)            (210.749741,)   
4              (12368, 0.140693)           (1740.089600,)   

  before_kernel_time_ratio after_num_worlds         after_allocating  \
0              (0.745825,)             (1,)      (2304, 25282281472)   
1              (0.859698,)          (1